In [45]:
from pathlib import Path
import torch

from IPython.display import clear_output

torch.__version__

'2.5.1'

In [46]:
data_dir = Path(r'F:\cc_data\yt_data')
output_locn = str(data_dir)

# Downloading Videos and Extracting Audio

## Option 1 - With PyTubeFix

In [47]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

In [51]:
import re

def replace_with_underscore(text):
    # Replace all non-alphanumeric characters (including spaces) with underscores
    return re.sub(r'\W+', '_', text)

url_list = ["https://www.youtube.com/watch?v=mQS1tCQUiAM",
            "https://www.youtube.com/watch?v=-OIi7O3cYAg",
            "https://www.youtube.com/watch?v=xutQXlphLiM",
            "https://www.youtube.com/watch?v=j9OHzRypRVs",
            "https://www.youtube.com/watch?v=RFiPUS9-cN0",
            "https://www.youtube.com/watch?v=iCXAT6ZZHuM"
           ]
download_list = []
title_list = []

for url in url_list:
    yt = YouTube(url, on_progress_callback = on_progress)
    
    title = replace_with_underscore(yt.title)
    print(yt.title, title)
    
    video_output = data_dir / f"{title}.mp4"

    try:
        ys = yt.streams.get_highest_resolution()
        ys.download(output_path = data_dir, filename = f"{title}")
        
        ys_audio = yt.streams.get_audio_only()
        ys_audio.download(output_path = data_dir, filename = f"{title}")

        download_list.append(video_output)
        title_list.append(title)

    except:
        print(f'Download failed for {title}')


How To Build A Monster Dividend Portfolio | Ep. 325 How_To_Build_A_Monster_Dividend_Portfolio_Ep_325
I Just Bought $30,000 Of This Stock I_Just_Bought_30_000_Of_This_Stock| 100.0%
We're Off To An Amazing Start We_re_Off_To_An_Amazing_Start███████████| 100.0%
5 Stocks To Buy And Hold Forever 5_Stocks_To_Buy_And_Hold_Forever█████| 100.0%
The Stock Market Has Become Overvalued The_Stock_Market_Has_Become_Overvalued%
I Just Broke A New Record I_Just_Broke_A_New_Record███████████████████| 100.0%


## Option 2 - PyTubeFix and MoviePy


In [52]:
from moviepy.editor import VideoFileClip

for title in title_list:

    video_output = data_dir / f"{title}.mp4"
    
    video_clip = VideoFileClip(str(video_output)) # purely to give us access to methods
    assert video_clip.audio is not None
    
    video_clip.audio.write_audiofile(str(data_dir / f"{title}.mp3"))
    video_clip.close()
    
    clear_output()

# Transcribing Audio to Text

In [53]:
for mp3_file in Path(output_locn).glob(f'*.mp3'):
    print(mp3_file)

F:\cc_data\yt_data\5_Stocks_To_Buy_And_Hold_Forever.mp3
F:\cc_data\yt_data\How_To_Build_A_Monster_Dividend_Portfolio_Ep_325.mp3
F:\cc_data\yt_data\I_Just_Bought_30_000_Of_This_Stock.mp3
F:\cc_data\yt_data\I_Just_Broke_A_New_Record.mp3
F:\cc_data\yt_data\The_Stock_Market_Has_Become_Overvalued.mp3
F:\cc_data\yt_data\v1.mp3
F:\cc_data\yt_data\We_re_Off_To_An_Amazing_Start.mp3


In [37]:
mp3_file_loc = data_dir / f"{title}.m4a"

if mp3_file_loc.exists():
    print("File exists")
else:
    print("File does not exist")

File exists


In [43]:
import speech_recognition as sr
from pydub import AudioSegment
import ffmpeg

def divide_chunks(sound, segment_size_secs=60):
    """ Split audio file into 60s chunks """
    
    segment_size_ms = segment_size_secs*1000
    for start_idx in range(0, len(sound), segment_size_ms):
        # Yield a chunk of audio data from start_idx to start_idx + segment_size_ms
        yield sound[start_idx:start_idx + segment_size_ms]

def transcribe_audio(mp3_file, title):
    """ Use Speech Recognition API with Google Web Speech API
    to convert audio dialog to text """
    recogniser = sr.Recognizer()      
    all_transcription_extracts = {}

    print(f"Converting {mp3_file}")

    audio = AudioSegment.from_file(mp3_file)
    # If AudioSegment is not working - e.g. due to broken mp3 headers - we
    # can use ffmpeg as a workaround. However, it's a lot slower.
    # ffmpeg.input(mp3_file).output(wav_file).run() # Convert with ffmpeg
    # print(f"Successfully converted {mp3_file} to {wav_file}.")
    # audio = AudioSegment.from_wav(wav_file) # Read the audio

    segments = list(divide_chunks(audio, segment_size_secs=60)) # split the wav into 60s segments     
    all_transcription_extracts[title] = {}
    for index, chunk in enumerate(segments):
        with io.BytesIO() as wav_io:
            chunk.export(wav_io, format='wav')
            wav_io.seek(0)  # Move to the start of the BytesIO object before reading from it
                    
            with sr.AudioFile(wav_io) as source:
                audio_data = recogniser.record(source)

            try:
                extracted = recogniser.recognize_google(audio_data, language=language)
                print(f"Chunk {index} extracted.")
                all_transcription_extracts[title][index] = extracted
            except sr.UnknownValueError:
                    # Log the unknown value error and continue
                print(f"Chunk {index}: Could not understand the audio. Maybe it was empty.")
        
    print("Extract:")
    for idx, extract in all_transcription_extracts[title].items():
        print(f"{idx}: {extract}")

    # Save as JSON file
    with open(output_locn/ f'{title}.json', 'w') as json_file:
        json.dump(all_transcription_extracts[title], json_file)

    # Save as TXT file
    with open(output_locn/ f'{title}.txt', 'w') as txt_file:
        for idx, extract in all_transcription_extracts[title].items():
            txt_file.write(f"{idx}: {extract}\n")

    # Save as CSV file using pandas
    df = pd.DataFrame(list(all_transcription_extracts[title].items()), columns=['Chunk', 'Transcription'])
    df.to_csv(output_locn / f'{title}.csv', index=False)


for mp3_file in Path(output_locn).glob(f'*.mp3'):
    transcribe_audio(str(mp3_file), )



Converting F:\cc_data\yt_data\v1.mp3
Chunk 0 extracted.
Chunk 1 extracted.
Chunk 2 extracted.
Chunk 3 extracted.
Chunk 4 extracted.
Chunk 5 extracted.
Chunk 6 extracted.
Chunk 7 extracted.
Chunk 8 extracted.
Chunk 9 extracted.
Chunk 10 extracted.
Chunk 11 extracted.
Chunk 12 extracted.
Chunk 13 extracted.
Chunk 14 extracted.
Chunk 15 extracted.
Chunk 16 extracted.
Chunk 17 extracted.
Chunk 18 extracted.
Chunk 19 extracted.
Chunk 20 extracted.
Chunk 21 extracted.
Chunk 22 extracted.
Chunk 23 extracted.
Chunk 24 extracted.
Chunk 25 extracted.
Chunk 26 extracted.
Chunk 27 extracted.
Chunk 28 extracted.
Chunk 29 extracted.
Chunk 30 extracted.
Chunk 31 extracted.
Chunk 32 extracted.
Chunk 33 extracted.
Chunk 34 extracted.
Chunk 35 extracted.
Chunk 36 extracted.
Chunk 37 extracted.
Chunk 38 extracted.
Chunk 39 extracted.
Chunk 40 extracted.
Chunk 41 extracted.
Chunk 42 extracted.
Chunk 43: Could not understand the audio. Maybe it was empty.
Extract:
0: everyone would love to have a monster d

In [44]:
transcription_extracts

NameError: name 'transcription_extracts' is not defined